# Classification of breast cancer using machine learning algorithms

Author:
* Augusto de Oliveira Perin
* Data Scientist
* Brazil

In [1]:
# Libraries Used
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


## Obtaining data from the Wisconsin Diagnostic Breast Cancer database

Creators:
* Dr. William H. Wolberg, General Surgery Dept., University of
Wisconsin, Clinical Sciences Center, Madison, WI 53792
wolberg@eagle.surgery.wisc.edu
* W. Nick Street, Computer Sciences Dept., University of
Wisconsin, 1210 West Dayton St., Madison, WI 53706
street@cs.wisc.edu 608-262-6619
* Olvi L. Mangasarian, Computer Sciences Dept., University of
Wisconsin, 1210 West Dayton St., Madison, WI 53706
olvi@cs.wisc.edu



In [2]:
# Data Source
data_cancer = pd.read_csv('https://query.data.world/s/tsceosr23236cbv6l63xscqoy7vrsn?dws=00000')

# Remove the last column
data_cancer = data_cancer.iloc[:, :-1]

In [3]:
# Displaying the Dataframe
data_cancer

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [4]:
# Modification of data to display below
data_cancer['diagnosis'] = data_cancer['diagnosis'].map({'B': 'Benign', 'M': 'Malignant'})

## Displaying class distribution

There are 357 instances diagnosed as benign and 212 diagnosed as
malignant

In [5]:
fig = px.pie(data_cancer, names='diagnosis', title='Class distribution')
fig.update_traces(textposition='inside', textinfo='percent+value+label')
fig.show()

## Principal Components Analysis

In [6]:
# Transform the diagnosis into Boolean values (0 for benign, 1 for malignant)
data_cancer['diagnosis'] = data_cancer['diagnosis'].map({'Benign': 0, 'Malignant': 1})

# Separate quantitative columns for PCA analysis
quantitative_columns = data_cancer.columns[1:]

In [7]:
# Normalize data
scaler = StandardScaler()
normalized_cancer_data = scaler.fit_transform(data_cancer[quantitative_columns])

# Perform PCA analysis
pca = PCA(n_components=2)
main_components = pca.fit_transform(normalized_cancer_data)

# Create a new dataframe with the PCA results
df_pca = pd.DataFrame(data=main_components, columns=['Main Component 1', 'Main Component 2'])

# Add the diagnostic and index columns to the new dataframe
df_pca[['id', 'diagnosis']] = data_cancer[['id', 'diagnosis']]

In [8]:
# View PCA scatterplot
# Plot the arrow chart
fig = go.Figure()

# Add additional information
proportion_explained = pca.explained_variance_ratio_
proportion_explained = np.round(proportion_explained, 2)

fig.add_trace(go.Scatter(
     x=df_pca['Main Component 1'],
     y=df_pca['Main Component 2'],
     mode='markers',
     marker=dict(
         color=df_pca['diagnosis'],
         colorscale=[[0, 'blue'], [1, 'red']],
         showscale=False
     ),
     text=df_pca['id'],
     hovertemplate='Index: %{text}<br>' +
                   'Diagnosis: %{marker.color}'
))

fig.update_layout(
     title='Principal Component Analysis',
     xaxis=dict(title='Main Component 1'),
     yaxis=dict(title='Main Component 2')
)

fig.add_annotation(
     text=f"Explained Proportion: {proportion_explained}",
     align='left',
     showarrow=False,
     xref='paper',
     yref='paper',
     x=0.05,
     y=0.9,
     bordercolor='black',
     borderwidth=1,
     borderpad=4,
     bgcolor='white',
     opacity=0.8,
     font=dict(size=18)
)

fig.show()

## Neural Network to find out if it is Malignant or Benign

In [9]:
# Split data into training and testing sets
X = data_cancer[quantitative_columns[1:]].values
y = data_cancer['diagnosis'].values

# Data separation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) # Training and Testing
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.40, random_state=42)

# Create the neural network
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate model performance
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model accuracy: {accuracy*100:.2f}%")

Epoch 1/50
14/14 [==============================] - 1s 9ms/step - loss: 69.9969 - accuracy: 0.3709 - val_loss: 49.3218 - val_accuracy: 0.3882
Epoch 2/50
14/14 [==============================] - 0s 2ms/step - loss: 40.0803 - accuracy: 0.3709 - val_loss: 25.6292 - val_accuracy: 0.3882
Epoch 3/50
14/14 [==============================] - 0s 2ms/step - loss: 17.3855 - accuracy: 0.3709 - val_loss: 6.1298 - val_accuracy: 0.3882
Epoch 4/50
14/14 [==============================] - 0s 2ms/step - loss: 3.6592 - accuracy: 0.4390 - val_loss: 1.4875 - val_accuracy: 0.6000
Epoch 5/50
14/14 [==============================] - 0s 2ms/step - loss: 1.0745 - accuracy: 0.5211 - val_loss: 0.3360 - val_accuracy: 0.8588
Epoch 6/50
14/14 [==============================] - 0s 2ms/step - loss: 0.4682 - accuracy: 0.8592 - val_loss: 0.2473 - val_accuracy: 0.9294
Epoch 7/50
14/14 [==============================] - 0s 2ms/step - loss: 0.3545 - accuracy: 0.8826 - val_loss: 0.1858 - val_accuracy: 0.9412
Epoch 8/50
14/1

In [10]:
# Make prediction using the trained model
prediction = model.predict_on_batch(X_valid)

# Create DataFrame with true class and neural network prediction
prediction = np.transpose(prediction)
prediction = np.reshape(prediction, [58])
prediction = prediction[:]
classification = np.array(prediction, copy=True)
classification[np.where(classification[:]> 0.5)] = 1
classification[np.where(classification[:]<= 0.5)] = 0
array_valid = np.stack((y_valid, prediction, classification), 1)
df_valid = pd.DataFrame(array_valid, columns = ['Class', 'Probability', 'Classification'])

# Creation of the hit column based on the correct classification
df_valid['Correctly'] = np.where(df_valid['Class'] == df_valid['Classification'], 1, 0)

# Creation of the column that describes the type of classification
classification_conditions = [
     (df_valid['Class'] == 1) & (df_valid['Classification'] == 1), # True positive
     (df_valid['Class'] == 0) & (df_valid['Classification'] == 0), # True negative
     (df_valid['Class'] == 1) & (df_valid['Classification'] == 0), # False negative
     (df_valid['Class'] == 0) & (df_valid['Classification'] == 1), # False positive
]
classification_types = ['True positive', 'True negative', 'False negative', 'False positive']
df_valid['Type'] = np.select(classification_conditions, classification_types)

# Display the DataFrame
df_valid

,Class,Probability,Classification,Correctly,Type
0,0.0,0.086811,0.0,1,True negative
1,1.0,1.000000,1.0,1,True positive
2,1.0,0.081018,0.0,0,False negative
3,1.0,0.999999,1.0,1,True positive
4,0.0,0.028660,0.0,1,True negative
5,0.0,0.033887,0.0,1,True negative
6,1.0,0.084450,0.0,0,False negative
7,0.0,0.041585,0.0,1,True negative
8,1.0,0.999968,1.0,1,True positive
9,0.0,0.138523,0.0,1,True negative


In [11]:
# create the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_valid, classification).ravel()
cm = confusion_matrix(y_valid, classification)

# create the heatmap
heatmap = go.Heatmap(z=cm, colorscale='Blues', x=['0', '1'], y=['0', '1'],
                     text=[['True negative', 'False positive'],
                     ['False negative', 'True positive']],texttemplate="%{text} <br> %{z}",
                    textfont={"size":20})

# create the layout
layout = go.Layout(title='Confusion Matrix', xaxis={'title': 'Predicted value'}, yaxis={'title': 'True value'})

# create the figure
fig = go.Figure(data=[heatmap], layout=layout)
# show the figure
fig.show()

In [12]:
# Calculation and addition of hit rate
rate = df_valid['Correctly'].sum() / df_valid.shape[0]
print(rate)

0.9310344827586207


## PCA with 3 Dimensions

In [13]:
# Perform PCA analysis
pca3d = PCA(n_components=3)
main_components3d = pca3d.fit_transform(normalized_cancer_data)

# Create a new dataframe with the PCA results
df_pca3d = pd.DataFrame(data=main_components3d, columns=['Main Component 1', 'Main Component 2', 'Main Component 3'])


# Add the diagnostic and index columns to the new dataframe
df_pca3d[['id', 'diagnosis']] = data_cancer[['id', 'diagnosis']]

# View PCA scatterplot
# Plot the arrow chart
fig = go.Figure()

# Add additional information
proportional_explained3d = pca3d.explained_variance_ratio_
proportion_explained3d = np.round(proportional_explained3d, 2)

fig.add_trace(go.Scatter3d(
     x=df_pca3d['Main Component 1'],
     y=df_pca3d['Main Component 2'],
     z=df_pca3d['Main Component 3'],
     mode='markers',
     marker=dict(
         color=df_pca3d['diagnosis'],
         colorscale=[[0, 'blue'], [1, 'red']],
         showscale=False
     ),
     text=df_pca3d['id'],
     hovertemplate='Index: %{text}<br>' +
                   'Diagnosis: %{marker.color}'
))

fig.add_annotation(
     text=f"Explained Proportion: {proportional_explained3d}",
     align='left',
     showarrow=False,
     xref='paper',
     yref='paper',
     x=0.05,
     y=0.9,
     bordercolor='black',
     borderwidth=1,
     borderpad=4,
     bgcolor='white',
     opacity=0.8,
     font=dict(size=18)
)

fig.update_layout(scene = dict(
                     xaxis_title='Main Component 1',
                     yaxis_title='Main Component 2',
                     zaxis_title='Main Component 3'),
                     width=700,
                     margin=dict(r=20, b=10, l=10, t=10))

fig.show()